In [22]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.cluster import KMeans
import json

In [29]:
FOLDER = "./../data/"
MAPPINGS_PATH = FOLDER + "mapping_gene_to_go.json"

#### Data preparation for clustering

In [44]:
gene_disease_csv = pd.read_csv(FOLDER+ "gene_disease_view.csv")
gene_disease_series = gene_disease_csv.groupby(["geneId"])["diseaseId"].apply(list)
gene_disease_dict = gene_disease_series.to_dict()

gene_to_list_of_diseases = []

#Could be inmproved propably somehow with pandas functionality
for gene_id, disease_ids in gene_disease_dict.items():
    new_dict = {'diseaseId': disease_ids, 'geneID': str(gene_id)}
    gene_to_list_of_diseases.append(new_dict)

C:\Users\samue\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Gene to GO mapping

In [35]:
if(os.path.isfile(MAPPINGS_PATH)):
    with open(MAPPINGS_PATH) as json_file:
        mapping_gene_to_go = json.load(json_file)

else:
    gaf_columns = [
        "DB", "DB_Object_ID", "DB_Object_Symbol", "Qualifier", "GO_ID",
        "DB:Reference", "Evidence_Code", "With_or_From", "Aspect", "DB_Object_Name",
        "DB_Object_Synonym", "DB_Object_Type", "Taxon", "Date", "Assigned_By"
    ]
    # ignoring first column, There were was only one value and it made as double index (atleast I think so).
    gaf_data = pd.read_csv(FOLDER + "goa_human.gaf", sep="\t", comment="!", names=gaf_columns, skiprows=12,usecols=range(1,16))
    mapping_gene_to_go = pd.Series(gaf_data.Qualifier.values,index=gaf_data.DB).to_dict()
    
    with open(MAPPINGS_PATH, "w") as fp:
        json.dump(mapping_gene_to_go, fp)  # encode dict into JSON


#### Feature extraction

In [45]:
vec = DictVectorizer()
matrixGeneDisease = vec.fit_transform(gene_to_list_of_diseases)
matrixGeneDisease

<21671x51841 sparse matrix of type '<class 'numpy.float64'>'
	with 3282995 stored elements in Compressed Sparse Row format>

#### Clustering

In [47]:
cluster = KMeans()
result = cluster.fit(matrixGeneDisease)
result.labels_

array([5, 5, 5, ..., 5, 5, 5])

#### TODO 
- extract geneID from clustering result, my guess is that labels_ are in same order as gene_to_list_of_diseases.
